# Пробуем простую полносвязную сеть

## Загрузка данных

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
plt.style.use('seaborn')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from skimage import transform
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import OrdinalEncoder
from read_data import *
from read_data import good_cols
%matplotlib inline

In [4]:
all_df = get_df()
TARGET_LEN = 2000
activities = sep_by_len(all_df, 20000)
cut_df = cut_act(activities, TARGET_LEN, count=-1)
n_df = normalize_df(cut_df)
n_df.sample()

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [00:02<00:00, 29.64it/s]


,activityID,heart rate,temperature hand,3Da_x scale_16 hand,3Da_y scale_16 hand,3Da_z scale_16 hand,3Dg_x hand,3Dg_y hand,3Dg_z hand,3Dm_x hand,...,temperature ankle,3Da_x scale_16 ankle,3Da_y scale_16 ankle,3Da_z scale_16 ankle,3Dg_x ankle,3Dg_y ankle,3Dg_z ankle,3Dm_x ankle,3Dm_y ankle,3Dm_z ankle
258,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.5882229755328096, 0.6032231717456451, 0.608...","[0.31839849974769785, 0.32468407716186826, 0.3...","[0.36020784063740896, 0.3456341699627997, 0.32...","[0.7388735050748416, 0.741253393221189, 0.7415...","[0.16393658626455507, 0.18148543736867515, 0.1...","[0.9018853243575603, 0.9043685581713764, 0.907...","[0.7359024286421003, 0.710113206244751, 0.6877...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.32330983194519913, 0.3060552849992206, 0.30...","[0.5226385366319242, 0.5247401281959164, 0.517...","[0.538872977193991, 0.530841019275478, 0.52599...","[0.40351536996127324, 0.414476175143921, 0.393...","[0.3972044490310744, 0.40070583139038585, 0.38...","[0.34744483701760015, 0.34236574292715904, 0.3...","[0.20206847707969536, 0.1601366730281616, 0.16...","[0.6217697915776631, 0.6091599322016386, 0.615...","[0.41692515784291706, 0.4169060398514533, 0.44..."


Получаем вектор признаков

In [5]:
X = n_df.iloc[:, 1:].values

Y = n_df.iloc[:, 0] # целевая переменная
X.shape, Y.shape

((783, 31), (783,))

In [6]:
X_flat = get_flatten(X)
X_flat.shape

(783, 62000)

In [12]:
y_targ = y_encode(Y)
np.unique(y_targ)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [8]:
X_train_tensor, X_val_tensor, y_train_tensor, y_val_tensor = \
    train_test_split(torch.FloatTensor(X_flat), torch.LongTensor(y_targ), random_state=42, test_size = 0.3)
X_train_tensor.shape, y_train_tensor.shape, np.unique(y_train_tensor), np.unique(y_val_tensor)

(torch.Size([548, 62000]),
 torch.Size([548]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64))

In [9]:
class SignalDataset(Dataset):
    def __init__(self, data_X, data_Y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = data_X
        self.Y = data_Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        input_data = self.X[idx]
        label = self.Y[idx]
        
        return input_data, label

In [10]:
BATCH_SIZE = 64

trainset = SignalDataset(X_train_tensor, y_train_tensor)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

testset = SignalDataset(X_val_tensor, y_val_tensor)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=0)
len(trainset)

548

## Подготовка модели

In [13]:
activation = nn.LeakyReLU

net = torch.nn.Sequential(
    nn.Linear(31*TARGET_LEN, 128),
    activation(),
    nn.Linear(128,128),
    activation(),
    nn.Linear(128,len(np.unique(y_targ)))
)

## Обучение

In [14]:
NUM_EPOCHS = 200

loss_fn = torch.nn.CrossEntropyLoss(size_average=False)
losses = []

learning_rate = 3e-3
optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
ebar = tqdm(range(NUM_EPOCHS))

for epoch_num in ebar:
    running_loss = 0.0
    iter_num = 0
#     pbar = tqdm(enumerate(trainloader), total = len(trainloader))
    pbar = enumerate(trainloader)
    for iter_num, (X_batch, y_batch) in pbar:
        iter_num +=1
        # forward (подсчёт ответа с текущими весами)
        y_pred = net(X_batch)
        
        # вычисляем loss'ы
        loss = loss_fn(y_pred, y_batch)
        running_loss += loss.item()
                     
        # зануляем градиенты
        optimizer.zero_grad()

        # backward (подсчёт новых градиентов)
        loss.backward()

        # обновляем веса
        optimizer.step()
        
    line = '[{}/{}] current loss: {}'.format(epoch_num, NUM_EPOCHS, running_loss/len(trainset))
    ebar.set_description(line)
    
    losses.append(running_loss/len(trainset))
        
    with torch.no_grad():
        y_pred = torch.softmax(net(X_val_tensor), 1)
        y = torch.argmax(y_pred, axis=1)
        line = '[{}/{}] current valid score: {}'.format(epoch_num+1, NUM_EPOCHS, f1_score(y_val_tensor, y, average='macro'))
        print(line)
#         ebar.set_description(line)

C:\Users\kitro\anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
[0/200] current loss: 25.292923586211934:   0%|▏                                       | 1/200 [00:00<02:46,  1.20it/s]

[1/200] current valid score: 0.03232323232323232


[1/200] current loss: 4.399809607624137:   1%|▍                                        | 2/200 [00:01<02:40,  1.24it/s]

[2/200] current valid score: 0.06198796918901187


[2/200] current loss: 2.507979107599189:   2%|▌                                        | 3/200 [00:02<02:35,  1.27it/s]

...

[198/200] current valid score: 0.5789224182349996


[198/200] current loss: 0.0002993197302694303: 100%|████████████████████████████████▊| 199/200 [02:27<00:00,  1.35it/s]

[199/200] current valid score: 0.5922857586150605


[199/200] current loss: 0.0002951318853582344: 100%|█████████████████████████████████| 200/200 [02:28<00:00,  1.35it/s]

[200/200] current valid score: 0.5900619862786376


In [15]:
fig, axs = plt.subplots(1,1,figsize=(20,8))
plt.plot(np.arange(NUM_EPOCHS), losses)
plt.show()

### Оценка точности

In [16]:
for i in range(len(np.unique(y_val_tensor))):
    idx = np.argwhere(y_val_tensor==i)[0]
    y_pred = torch.sigmoid(net(X_val_tensor[idx]))
    y = torch.argmax(y_pred, axis=1)
    print(np.unique(Y)[i], len(y[y==i])/len(idx))

1 0.6086956521739131
2 0.3333333333333333
3 0.45161290322580644
4 0.7
5 0.7777777777777778
6 0.8888888888888888
7 0.76
16 0.375
17 0.5714285714285714


In [17]:
with torch.no_grad():
    y_pred = torch.softmax(net(X_val_tensor), 1)
    y = torch.argmax(y_pred, axis=1)
#     print(net(X_val_tensor).shape)
    print(roc_auc_score(y_val_tensor, y_pred, multi_class='ovo'))
    print(f1_score(y_val_tensor, y, average='macro'))

0.9046193503849421
0.5900619862786376
